***GENERATED CODE FOR forecastingmilkproduction PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
from pyspark.sql.functions import col, udf, round
from pyspark.ml.feature import Normalizer
from pyspark.sql.types import *
import json
from pyspark.sql.functions import col, round
from pyspark.ml.feature import StandardScaler
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def normalizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']

    dfReturn = dfReturn.fillna({feature: '0.0'})

    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature) \
        .withColumnRenamed("feature_cast", feature)

    outcol = feature + "_normalizer"
    p = transformationData["pNorm"]

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    normalizer = Normalizer(inputCol=featureVector,
                            outputCol=outcol, p=float(p))
    normalized = normalizer.transform(dfReturn)

    dfReturn = normalized.withColumn("final_col", to_array(normalized[outcol])) \
        .select(normalized.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False

    outcol = feature + "_standardscalar"

    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))

    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = normalizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Year_Month', 'pNorm': '2000.00', 'transformation_label': 'Normalizer'}], 'feature': 'Year_Month', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                               'count': '62', 'mean': '2004.16', 'stddev': '5.74', 'min': '1995.07', 'max': '2013.12', 'missing': '0'}, 'transformation': [{'transformation': 'Normalizer', 'selectedAsDefault': 1}]}, {'feature_label': 'Year_Month', 'pNorm': '2000.00', 'transformation_label': 'Normalizer'})
        transformationDF = transformationDF.drop('Year_Month')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'Month_Number', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'Month_Number', 'transformation': [
            {'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {'count': '62', 'mean': '6.45', 'stddev': '3.69', 'min': '1', 'max': '12', 'missing': '0'}}, {'feature_label': 'Month_Number', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'False'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('Month_Number')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Month', 'transformation_label': 'String Indexer'}], 'feature': 'Month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '62', 'mean': '', 'stddev': '', 'min': 'Apr', 'max': 'Sept', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Month', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Month')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run forecastingmilkproductionHooks.ipynb
try:
	sourcePreExecutionHook()

	forecastingmilkproductiondbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/MilkProductionTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(forecastingmilkproductiondbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run forecastingmilkproductionHooks.ipynb
try:
	transformationPreExecutionHook()

	forecastingmilkproductionautofe = TransformationMain.run(forecastingmilkproductiondbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Year_Month", "pNorm": "2000.00", "transformation_label": "Normalizer"}], "feature": "Year_Month", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "62", "mean": "2004.16", "stddev": "5.74", "min": "1995.07", "max": "2013.12", "missing": "0"}, "transformation": [{"transformation": "Normalizer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Month_Number", "std_scalar": {"mean_flag": "True", "std_flag": "False"}, "transformation_label": "Standard Scalar"}], "feature": "Month_Number", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "62", "mean": "6.45", "stddev": "3.69", "min": "1", "max": "12", "missing": "0"}}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "62", "mean": "2004.1", "stddev": "5.75", "min": "1995", "max": "2013", "missing": "0"}}, {"transformationsData": [{"feature_label": "Month", "transformation_label": "String Indexer"}], "feature": "Month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "62", "mean": "", "stddev": "", "min": "Apr", "max": "Sept", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Cotagecheese_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "62", "mean": "2.84", "stddev": "0.53", "min": "1.941", "max": "4.377", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Icecream_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "62", "mean": "70.75", "stddev": "11.28", "min": "50.85", "max": "90.472", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Milk_Prod", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "62", "mean": "2.94", "stddev": "0.49", "min": "2.03", "max": "3.744", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "N_CA_Fat_Price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "62", "mean": "1.56", "stddev": "0.39", "min": "0.8934", "max": "2.4996", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"feature": "Month_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "62", "mean": "4.5", "stddev": "3.47", "min": "0.0", "max": "13.0", "missing": "0"}}, {"feature": "Year_Month_normalizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "62", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.0", "missing": "0"}}, {"feature": "Month_Number_standardscalar_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "selected": "True", "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "stats": {"count": "62", "mean": "-0.0", "stddev": "3.69", "min": "-5.451612903225808", "max": "5.548387096774192", "missing": "0"}}]}))

	transformationPostExecutionHook(forecastingmilkproductionautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run forecastingmilkproductionHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionRegression(forecastingmilkproductionautofe, ["Year", "Cotagecheese_Prod", "Icecream_Prod", "Milk_Prod", "Month_stringindexer", "Year_Month_normalizer", "Month_Number_standardscalar"], "N_CA_Fat_Price")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

